# Building a Spam Filter with Naive Bayes


In this project, we're going to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. **Our goal is to write a program that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).**

To train the algorithm, we'll use a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](<https://archive.ics.uci.edu/ml/datasets/sms+spam+collection>). The data collection process is described in more details on [this page](<http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition>), where you can also find some of the papers authored by Tiago A. Almeida and José María Gómez Hidalgo.

## Read and Explore the Dataset

In [1]:
#reading dataset
import pandas as pd

smsdata = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

In [2]:
#Find how many rows and columns it has
print(smsdata.shape)
smsdata.head()

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
#Finding what percentage of the messages is spam and what percentage is ham ("ham" means non-spam).
round(smsdata['Label'].value_counts(normalize=True) * 100, 0)

ham     87.0
spam    13.0
Name: Label, dtype: float64

Above output reveals that 87% of the messages are ham ("ham" means non-spam), and the remaining 13% are spam in the dataset.

## Training Set and Test Set

Now that we've become a bit familiar with the dataset, we can move on to building the spam filter.

However, before creating it, it's very helpful to first think of a way of testing how well it works. When creating software (a spam filter is software), a good rule of thumb is that designing the test comes before creating the software. If we write the software first, then it's tempting to come up with a biased test just to make sure the software passes it.

Once our spam filter is done, we'll need to test how good it is with classifying new messages. To test the spam filter, we're first going to split our dataset into two categories:

* A training set, which we'll use to "train" the computer how to classify messages.
* A test set, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:

* The training set will have 4,458 messages (about 80% of the dataset).
* The test set will have 1,114 messages (about 20% of the dataset).

To better understand the purpose of putting a test set aside, let's begin by observing that all 1,114 messages in our test set are already classified by a human. When the spam filter is ready, we're going to treat these messages as new and have the filter classify them. Once we have the results, we'll be able to compare the algorithm classification with that done by a human initially, and this way we'll see how good the spam filter really is.

As mentioned before, our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam) with our training set.

In [4]:
#We're going to start by randomizing the entire dataset to ensure that spam and ham messages are spread properly throughout
#the dataset.

In [5]:
#Using the frac=1 parameter to randomize the entire dataset.
#Using the random_state=1 parameter to make sure the results are reproducible.
smsdatapreparation = smsdata.sample(frac=1, random_state=1)

In [6]:
#Splitting the above randomized dataset into a training and a test set. The training set should account for 80% of the dataset,
#as mentioned earlier and the remaining 20% of the data should be the test set

In [7]:
training_set_index = round(len(smsdatapreparation) * 0.8)

training_set = smsdatapreparation[:training_set_index].reset_index(drop=True)
test_set = smsdatapreparation[training_set_index:].reset_index(drop=True)

In [8]:
#Checking that the training set have 4,458 messages (about 80% of the dataset) and 
#the test set have 1,114 messages (about 20% of the dataset).

In [9]:
print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


In [10]:
#Finding the percentage of spam and ham in both the training set and the test set

In [11]:
round(training_set['Label'].value_counts(normalize=True) * 100, 0)

ham     87.0
spam    13.0
Name: Label, dtype: float64

In [12]:
#above output shows our training set is a perfect representative of the dataset.

In [13]:
round(test_set['Label'].value_counts(normalize=True) * 100, 0)

ham     87.0
spam    13.0
Name: Label, dtype: float64

In [14]:
#all seems good to move on.

## Use the training set to teach the algorithm to classify new messages.

When a new message comes in, our Naive Bayes algorithm will make the classification based on the results it gets through these two equations:

`P(Spam|w1,w2,...,wn) ∝ P(Spam) ⋅ ∏i=1nP(wi|Spam)`

`P(Ham|w1,w2,...,wn) ∝ P(Ham) ⋅ ∏i=1nP(wi|Ham)`

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we need to use these equations:

`P(wi|Spam)=Nwi|Spam + α / NSpam + α ⋅ NVocabulary`

`P(wi|Ham)=Nwi|Ham + α / NHam + α ⋅ NVocabulary`

Summarizing what the terms in the equations above mean:

* Nwi|Spam=the number of times the word wi occurs in Spam messages
* Nwi|Ham=the number of times the word wi occurs in Ham messages
* NSpam=total number of words in spam messages
* NHam=total number of words in non-spam messages
* NVocabulary=total number of words in the vocabulary
* α=1    (α is a smoothing parameter)

To calculate all these probabilities, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need. 

Right now, our training and test sets have this format:

In [15]:
training_set.columns

Index(['Label', 'SMS'], dtype='object')

In [16]:
test_set.columns

Index(['Label', 'SMS'], dtype='object')

In [17]:
training_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [18]:
test_set.head()

,Label,SMS
0,ham,Later i guess. I needa do mcat study too.
1,ham,But i haf enuff space got like 4 mb...
2,spam,Had your mobile 10 mths? Update to latest Oran...
3,ham,All sounds good. Fingers . Makes it difficult ...
4,ham,"All done, all handed in. Don't know if mega sh..."


To make the calculations easier, we want bring the data to the format by cleaning and transforming:

* The SMS column will not exist anymore, instead the SMS column will be replaced by a series of new columns, where each column represents a unique word from the vocabulary.
* Each row will describe a single message and in the vocabulary word columns it will have the values representing the number of times these words are present in that message (frequency).
* All words in the vocabulary are in lower case for uniformity, so the message in our dataset should too be in lower case.
* Punctuation will not be taken into account.

### Data Cleaning

In [19]:
#The data cleaning process by removing the punctuation and bringing all the words to lower case.

In [20]:
#Removing all the punctuation from the SMS column by using the regex '\W' 
#to detect any character that is not from a-z, A-Z or 0-9.
#For instance, the function re.sub('\W', ' ', 'Secret!! Money, goods.' ) strips all the punctuation marks and 
#outputs the string as 'Secret Money goods '.
#For simplicity, we will be using the Series.str.replace() method here instead for removing all the punctuation and 
#Series.str.lower() method for transforming every letter in every word to lower case.

In [21]:
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### Creating the Vocabulary  -  a set of unique words

In [22]:
#Creating a vocabulary for the messages in the training set.

In [23]:
#Transforming each message from the SMS column into a list by splitting the string at the space character

In [24]:
training_set['SMS'] = training_set['SMS'].str.split()

In [25]:
#Creating a Python list containing all the unique words across all messages, where each word is represented as a string.

In [26]:
vocabulary = []
for message in training_set['SMS']:
    for word in message:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary))
vocabulary

['armenia',
 'gbp',
 'prem',
 'tscs',
 'fated',
 'hp',
 'maximum',
 '27',
 'sugababes',
 'misss',
 'best',
 'quizclub',
 'bmw',
 'places',
 'picsfree1',
 'worlds',
 'snowman',
 'tsandcs',
 'w1t1jy',
 'nat',
 '08719181259',
 'lotto',
 'devils',
 'ntt',
 '50award',
 'heal',
 'lookin',
 'gdeve',
 'transfr',
 'packing',
 'ymca',
 'sisters',
 'hills',
 'than',
 'lager',
 'happily',
 '08718726970',
 'scenario',
 'nike',
 '087123002209am',
 'yoyyooo',
 'vpod',
 'openin',
 'muht',
 'das',
 'exams',
 'type',
 'compulsory',
 'nokia6650',
 'colleagues',
 'quiet',
 'la',
 'commercial',
 'invnted',
 'deliveredtomorrow',
 'pshew',
 'anniversary',
 'syria',
 'mtmsg',
 'haul',
 'songs',
 'cheered',
 'skyped',
 'choose',
 'polyc',
 'goodmate',
 'coat',
 'weather',
 'll',
 'resubbing',
 'wasted',
 'fingers',
 'slip',
 'glorious',
 '087187262701',
 'goodnoon',
 'indian',
 'webeburnin',
 'nange',
 'zaher',
 'cousin',
 'solve',
 'thats',
 'sub',
 'nursery',
 '09058091870',
 '1250',
 'bawling',
 'php',
 'di

In [27]:
#total numbers of unique words in vocabulary
len(vocabulary)

7783

### Use the vocabulary to make the data transformation

In [28]:
#Creating a new DataFrame

In [29]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [30]:
vocabularydata = pd.DataFrame(word_counts_per_sms)
vocabularydata.head()

,armenia,gbp,prem,tscs,fated,hp,maximum,27,sugababes,misss,...,released,away,surprise,08714712388,bx526,85023,hav2hear,scraped,165,pleasant
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
#Concatenating the DataFrame we just built above with the DataFrame containing the training set 

In [32]:
final_training_set = pd.concat([training_set, vocabularydata], axis=1)
final_training_set.head()

,Label,SMS,armenia,gbp,prem,tscs,fated,hp,maximum,27,...,released,away,surprise,08714712388,bx526,85023,hav2hear,scraped,165,pleasant
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Creating the spam filter

We had discuss 4 equations above to built our spam filter, let us just revise once more:

`P(Spam|w1,w2,...,wn) ∝ P(Spam) ⋅ ∏i=1nP(wi|Spam)`

`P(Ham|w1,w2,...,wn) ∝ P(Ham) ⋅ ∏i=1nP(wi|Ham)`

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we need to use these equations:

`P(wi|Spam)=Nwi|Spam + α / NSpam + α ⋅ NVocabulary`

`P(wi|Ham)=Nwi|Ham + α / NHam + α ⋅ NVocabulary`

Now some of the terms in the four equations above will have the same value for every new message. As a start, let's first calculate:

* P(Spam) and P(Ham)
* NSpam, NHam, NVocabulary

    * NSpam is equal to the number of words in all the spam messages — it's not equal to the number of spam messages, and it's not equal to the total number of unique words in spam messages.
    * NHam is equal to the number of words in all the non-spam messages — it's not equal to the number of non-spam messages, and it's not equal to the total number of unique words in non-spam messages.
    * We'll also use Laplace smoothing and set α=1. 

In [33]:
#Using the training set only:

#Calculating P(Spam) and P(Ham)

In [34]:
#Isolating spam and ham messages data first
spam_messages = final_training_set[final_training_set['Label'] == 'spam']
ham_messages = final_training_set[final_training_set['Label'] == 'ham']

#P(Spam) and P(Ham)
p_spam = int(round((len(spam_messages) / len(final_training_set)) * 100, 0))
p_ham = int(round((len(ham_messages) / len(final_training_set)) * 100, 0))
print(p_spam)
print(p_ham)

13
87


In [35]:
#Calculate NSpam, NHam, NVocabulary
#N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()
print(n_spam)

#N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()
print(n_ham)

#N_Vocabulary
n_vocabulary = len(vocabulary)
print(n_vocabulary)

#Initializing a variable named alpha with a value of 1
#Laplace smoothing
alpha = 1
print(alpha)

15190
57237
7783
1


As already mentioned earlier, all the above terms will have constant values in our equations for every new message (regardless of the message or each individual word in the message).

However, P(wi|Spam) and P(wi|Ham) will vary depending on the individual words. Although both P(wi|Spam) and P(wi|Ham) vary depending on the word, but the probability for each individual word is constant for every new message.

We will be using the training set to get the values we need to find a result for both the equations below:

`P(wi|Spam) = Nwi|Spam + α / NSpam + α ⋅ NVocabulary`

`P(wi|Ham) = Nwi|Ham + α / NHam + α ⋅ NVocabulary`

The steps we take to calculate P(wi|Spam) or P(wi|Ham) will be identical for every other new messages that contains the word `wi`. The key detail here is that calculating P(wi|Spam) or P(wi|Ham) only depends on the training set, and as long as we don't make changes to the training set, P(wi|Spam) or P(wi|Ham) stays constant. This means that we can use our training set to calculate the probability for both Spam and Ham of each word in our vocabulary. 

We have 7,783 words in our vocabulary, which means we'll need to calculate a total of 15,566 probabilities, as for each word we need to calculate both P(wi|Spam) and P(wi|Ham).

In more technical language, the probability values that P(wi|Spam) and P(wi|Ham) will take are called parameters.

The fact that we calculate so many values even before beginning with the classification of new messages makes the Naive Bayes algorithm very fast (especially compared to other algorithms). When a new message comes in, most of the needed computations are already done, which enables the algorithm to almost instantly classify the new message.

If we didn't calculate all these values beforehand, then all these calculations would need to be done every time a new message comes in. Imagine the algorithm to be used to classify 1,000,000 new messages. Why repeat all these calculations 1,000,000 times when we could just do them once at the beginning?

Let's now calculate all the parameters using the equations below:
 
`P(wi|Spam) = Nwi|Spam + α / NSpam + α ⋅ NVocabulary`

`P(wi|Ham) = Nwi|Ham + α / NHam + α ⋅ NVocabulary`

As P(wi|Spam) and P(wi|Ham) are key parts of the equations that we need to classify the new messages:

`P(Spam|w1,w2,...,wn) ∝ P(Spam) ⋅ ∏i=1nP(wi|Spam)`

`P(Ham|w1,w2,...,wn) ∝ P(Ham) ⋅ ∏i=1nP(wi|Ham)`

### Calculating Parameters

In [36]:
#Initializing two dictionaries, one dictionary to store the parameters for P(wi|Spam) and the other for P(wi|Ham), 
#where each key-value pair is a unique word (from our vocabulary) represented as a string and the value is 0.

In [37]:
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

In [38]:
# Calculating parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

Now that we've calculated all the constants and parameters we require, we can start creating the spam filter. The spam filter can be understood as a function that:

* Takes in as input a new message (w1, w2, ..., wn)
* Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
* Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
    * If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
    * If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
    * If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

Just a little briefing how our spam filter function (classify() function) works:

* The input variable message is assumed to be a string.
* We perform a bit of data cleaning on the string message:
    * We remove the punctuation using the re.sub() function.
    * We bring all letters to lower case using the str.lower() method.
    * We split the string at the space character and transform it into a Python list using the str.split() method.
* Calculate p_spam_given_message and p_ham_given_message:

    `P(Spam|w1,w2,...,wn) ∝ P(Spam) ⋅ ∏i=1nP(wi|Spam)`
    
    `P(Ham|w1,w2,...,wn) ∝ P(Ham) ⋅ ∏i=1nP(wi|Ham)`


* We compare p_spam_given_message with p_ham_given_message and then print a classification label.
* If some new messages will contain words that are not part of the vocabulary, then we will simply ignore these words when we're calculating the probabilities.

In [39]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [40]:
#classifying two new messages.

In [41]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3021578215095486e-23
P(Ham|message): 1.947076294334492e-25
Label: Spam


In [42]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.354127765568133e-23
P(Ham|message): 3.7070863895712615e-19
Label: Ham


## Measuring the Spam Filter's Accuracy

The two outputs above look promising, but let's see how well the filter does on our test set, which has 1,114 messages.

The algorithm will output a classification label for every message in our test set, which we'll be able to compare with the actual label (given by a human). Note that, in training, our algorithm didn't see these 1,114 messages, so every message in the test set is practically new from the perspective of the algorithm.

First off, we'll change the classify() function that we wrote previously to return the labels instead of printing them. Below, note that we now have return statements instead of print() functions:

In [43]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Now that we have a function that returns labels instead of printing them, we can use it to create a new column in our test set.

In [44]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now we will compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use accuracy as a metric:

Accuracy = number of correctly classified messages / total number of classified messages

In [45]:
#Measuring the accuracy of our spam filter.

In [46]:
correct = 0
total = len(test_set)
    
for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
accuracy= round((correct/total)*100, 2)
print(accuracy)

98.74


With the above output of 98.74%, our spam filter seems functionally really good.

## Concluding

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set, which is an excellent result. We initially aimed for an accuracy of over 80%, but we managed to do way better than that.